# ***Bibliotecas***

In [ ]:
import pygame
import time
import random
import numpy as np
import math
import matplotlib.pyplot as plt

# **Dados Iniciais**

In [ ]:
pygame.init()

# Configurações da tela
largura = 800
altura = 400
tela = pygame.display.set_mode((largura, altura))
pygame.display.set_caption('Jogo da Cobrinha')


# Cores
preto = (0, 0, 0)
verde = (0, 255, 0)
vermelho = (255, 0, 0)
azul = (0, 0, 255)
branco = (255, 255, 255)


# Tamanho da cobra e da maçã
tamanho_cobra = 10
velocidade_cobra = 15
espessura_parede = 1  # Define a espessura da parede


# **Q- Learning(IA)**

In [ ]:
# Q-learning parâmetros
alpha = 0.1
gamma = 0.9
epsilon = 1.0
epsilon_min = 0.01
epsilon_decay = 0.995

# Número de cobras no jogo
num_cobras = 10
q_tables = [{} for _ in range(num_cobras)]

# Cores das Cobras aleatórias
cores_cobras = [(random.randint(0, 225), random.randint(0, 225), random.randint(0, 225)) for _ in range(num_cobras)]

# **Features**

In [ ]:
#Função para desenhar a cobra
def desenhar_cobra(tamanho_cobra, lista_cobra, cor):
    for x in lista_cobra:
        pygame.draw.rect(tela, cor, [x[0], x[1], tamanho_cobra, tamanho_cobra])


# Função para exibir a pontuação
def mostrar_pontuacao(pontuacao, epoca):
    fonte = pygame.font.SysFont("Castellar", 25)
    texto = fonte.render("Pontuação: " + str(pontuacao) + " - Época: " + str(epoca), True, branco)
    tela.blit(texto, [0, 0])


# Função que gera ações possíveis
def escolher_acao(estado, i):
    if np.random.uniform(0, 1) < epsilon:
        return np.random.choice(['LEFT', 'RIGHT', 'UP', 'DOWN'])
    else:
        q_table = q_tables[i].get(estado, {'LEFT': 0, 'RIGHT': 0, 'UP': 0, 'DOWN': 0})
        return max(q_table, key=q_table.get)

# Função para calcular a distância da cobra até a comida
def calcular_distancia(x1, y1, comida_x, comida_y):
    return np.sqrt((x1 - comida_x)**2 + (y1 - comida_y)**2)


# Função para desenhar a parede
def desenhar_paredes():
    pygame.draw.rect(tela, vermelho, [0, 0, largura, espessura_parede])  # Parede superior
    pygame.draw.rect(tela, vermelho, [0, altura - espessura_parede, largura, espessura_parede])  # Parede inferior
    pygame.draw.rect(tela, vermelho, [0, 0, espessura_parede, altura])  # Parede esquerda
    pygame.draw.rect(tela, vermelho, [largura - espessura_parede, 0, espessura_parede, altura])  # Parede direita


# Função para calcular a distância entre dois pontos
def calcular_distancia(x1, y1, x2, y2):
    return ((x2 - x1)**2 + (y2 - y1)**2)**0.5


# **Jogo Completo**

In [ ]:
# Função principal do jogo
def jogo():
    global q_tables, epsilon
    epoca = 0  # Inicializa a contagem de épocas
    comida_inicial_x = largura / 4  # Posição fixa para a primeira comida
    comida_inicial_y = altura / 4

    pontuacoes = []  # Lista para armazenar as pontuações das épocas

    while True:
        # Inicializa as cobras e a comida
        cobras = [{'x1': largura / 2, 'y1': altura / 2, 'x1_mover': 0, 'y1_mover': 0,
                   'lista_cobra': [], 'comprimento_cobra': 1, 'cor': cores_cobras[i],
                   'tempo_sobrevivencia': time.time()} for i in range(num_cobras)]

        # Primeira comida em posição fixa, outras aleatórias
        comida_x = comida_inicial_x
        comida_y = comida_inicial_y

        inicio_epoca = time.time()  # Inicializa o temporizador
        clock = pygame.time.Clock()  # Controla a taxa de atualização
        game_over = False  # Flag para o estado do jogo
        cobra_morreu = [False] * num_cobras  # Lista para verificar se cada cobra morreu

        while not game_over:
            tempo_decorrido = time.time() - inicio_epoca
            if tempo_decorrido > 60:
                game_over = True

            for evento in pygame.event.get():
                if evento.type == pygame.QUIT:
                    pygame.quit()
                    quit()

            for i, cobra in enumerate(cobras):
                estado_anterior = (cobra['x1'], cobra['y1'], comida_x, comida_y)

                # Inicializa a Q-table para o estado anterior se não estiver presente
                if estado_anterior not in q_tables[i]:
                    q_tables[i][estado_anterior] = {'LEFT': 0, 'RIGHT': 0, 'UP': 0, 'DOWN': 0}

                # Escolha de ação
                acao = escolher_acao(estado_anterior, i)

                # Atualiza movimento da cobra
                if acao == 'LEFT':
                    cobra['x1_mover'] = -tamanho_cobra
                    cobra['y1_mover'] = 0
                elif acao == 'RIGHT':
                    cobra['x1_mover'] = tamanho_cobra
                    cobra['y1_mover'] = 0
                elif acao == 'UP':
                    cobra['y1_mover'] = -tamanho_cobra
                    cobra['x1_mover'] = 0
                elif acao == 'DOWN':
                    cobra['y1_mover'] = tamanho_cobra
                    cobra['x1_mover'] = 0

                cobra['x1'] += cobra['x1_mover']
                cobra['y1'] += cobra['y1_mover']

                # Checa colisão com as paredes
                if (cobra['x1'] >= largura or cobra['x1'] < 1 or
                    cobra['y1'] >= altura or cobra['y1'] < 1):
                    cobra['morreu'] = True
                else:
                    cobra['morreu'] = False

                # Checa colisão com si mesma
                cabeca_cobra = [cobra['x1'], cobra['y1']]
                cobra['lista_cobra'].append(cabeca_cobra)
                if len(cobra['lista_cobra']) > cobra['comprimento_cobra']:
                    del cobra['lista_cobra'][0]
                for segmento in cobra['lista_cobra'][:-1]:
                    if segmento == cabeca_cobra:
                        cobra['morreu'] = True

                # Adiciona recompensa por sobrevivência quando a cobra morre
                if cobra['morreu']:
                    cobra_morreu[i] = True
                    tempo_sobrevivencia = time.time() - cobra['tempo_sobrevivencia']
                    recompensa = tempo_sobrevivencia * 0.1  # Ajuste o fator de recompensa conforme necessário
                    estado_novo = (cobra['x1'], cobra['y1'], comida_x, comida_y)
                    if estado_novo not in q_tables[i]:
                        q_tables[i][estado_novo] = {'LEFT': 0, 'RIGHT': 0, 'UP': 0, 'DOWN': 0}
                    max_q_proximo_estado = max(q_tables[i].get(estado_novo, {'LEFT': 0, 'RIGHT': 0, 'UP': 0, 'DOWN': 0}).values())
                    for acao in ['LEFT', 'RIGHT', 'UP', 'DOWN']:
                        q_tables[i][estado_anterior][acao] = (1 - alpha) * q_tables[i][estado_anterior][acao] + \
                                                              alpha * (recompensa + gamma * max_q_proximo_estado)
                    continue

                # Checa se a cobra comeu a comida
                if cobra['x1'] == comida_x and cobra['y1'] == comida_y:
                    comida_x = round(random.randrange(0, largura - tamanho_cobra) / 10.0) * 10.0
                    comida_y = round(random.randrange(0, altura - tamanho_cobra) / 10.0) * 10.0
                    cobra['comprimento_cobra'] += 1
                    recompensa = 30
                else:
                    # Calcula a distância e ajusta a recompensa
                    distancia = calcular_distancia(cobra['x1'], cobra['y1'], comida_x, comida_y)
                    recompensa = -distancia / (largura + altura) * 10  # Ajusta a recompensa com base na distância

                estado_novo = (cobra['x1'], cobra['y1'], comida_x, comida_y)
                if estado_novo not in q_tables[i]:
                    q_tables[i][estado_novo] = {'LEFT': 0, 'RIGHT': 0, 'UP': 0, 'DOWN': 0}

                # Atualiza a Q-table
                max_q_proximo_estado = max(q_tables[i].get(estado_novo, {'LEFT': 0, 'RIGHT': 0, 'UP': 0, 'DOWN': 0}).values())
                for acao in ['LEFT', 'RIGHT', 'UP', 'DOWN']:
                    q_tables[i][estado_anterior][acao] = (1 - alpha) * q_tables[i][estado_anterior][acao] + \
                                                          alpha * (recompensa + gamma * max_q_proximo_estado)

            tela.fill(preto)  # Limpa a tela
            desenhar_paredes()  # Desenha as paredes

            # Desenha as cobras e a comida
            for cobra in cobras:
                if not cobra.get('morreu', False):
                    desenhar_cobra(tamanho_cobra, cobra['lista_cobra'], cobra['cor'])
            pygame.draw.rect(tela, azul, [comida_x, comida_y, tamanho_cobra, tamanho_cobra])
            mostrar_pontuacao(max(cobra['comprimento_cobra'] - 1 for cobra in cobras), epoca)

            pygame.display.update()  # Atualiza a tela
            clock.tick(velocidade_cobra)  # Controla FPS

            # Verifica se todas as cobras morreram e reinicia a época
            if all(cobra_morreu) or tempo_decorrido > 60:
                pontuacoes.append(max(cobra['comprimento_cobra'] - 1 for cobra in cobras))
                epoca += 1


                # Atualiza o epsilon após cada época
                epsilon = max(epsilon * epsilon_decay, epsilon_min)

                # Mensagem de atualização
                print(f'Época Atual: {epoca}')
                print(f'Maior Pontuação: {max(pontuacoes)}')
                print(f"Estado Anterior: {estado_anterior}")
                print(f"Estado Novo: {estado_novo}")
                print(f"Recompensa: {recompensa}")
                print(f"Q-table: {q_tables[i]}")
                print('#' *50)

                break  # Sai do loop do jogo para iniciar uma nova época

In [ ]:
tempo_init = time.time()
tempo_decorrido = time.time() - tempo_init

print(f"Tempo decorrido: {tempo_decorrido:.2f} segundos")
time.sleep(1)  # Atualiza a cada 1 segundo
jogo()


Tempo decorrido: 0.00 segundos


KeyboardInterrupt: 